# Final Project
Wil Gaboury and Calvin Hirsch

In [73]:
import pandas as pd
import warnings

In [74]:
csgo_df = pd.read_csv('./data/csgo.csv')
csgo_df = csgo_df[pd.to_numeric(csgo_df.best_of, errors='coerce').notnull()]
csgo_df

/home/wil/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,date,match_id,event_id,team_1,team_2,best_of,_map,t1_start,t2_start,1_t1,...,21_winner,22_winner,23_winner,24_winner,25_winner,26_winner,27_winner,28_winner,29_winner,30_winner
0,2020-03-01,2339402,4901,G2,Natus Vincere,5,Nuke,t,ct,4350.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-01,2339402,4901,G2,Natus Vincere,5,Dust2,ct,t,3900.0,...,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,NaN
2,2020-03-01,2339402,4901,G2,Natus Vincere,5,Mirage,t,ct,4150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Dust2,t,ct,4150.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Nuke,ct,t,4200.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43229,2017-04-04,2309263,2683,FaZe,HellRaisers,1,Cache,t,ct,4250.0,...,2.0,1.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN
43230,2017-04-04,2309262,2683,Astralis,CLG,1,Train,t,ct,4250.0,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN
43231,2017-04-04,2309261,2683,SK,TYLOO,1,Mirage,ct,t,4350.0,...,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,NaN
43232,2017-04-04,2309260,2683,Virtus.pro,MVP Project,1,Mirage,t,ct,4150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
csgo_df['win_ratio'] = csgo_df.loc[:, '1_winner':'30_winner'].apply(lambda x: len(x[x == 1]) - len(x[x == 2]), axis=1)
csgo_df.loc[:, '1_winner':'30_winner'] = csgo_df.loc[:, '1_winner':'30_winner'].applymap(lambda x: True if x == 1 else False if x == 2 else None)

csgo_df['t1_start_ct'] = csgo_df['t1_start'].apply(lambda x: x == 'ct')
csgo_df = csgo_df.drop(['t1_start', 't2_start'], axis=1)

csgo_df = csgo_df.rename(columns={'_map':'map'})

csgo_df[['t1_start_ct', '1_winner', '30_winner', 'map', 'win_ratio']]

,t1_start_ct,1_winner,30_winner,map,win_ratio
0,False,False,None,Nuke,-12
1,True,False,None,Dust2,-3
2,False,False,None,Mirage,-14
3,False,True,None,Dust2,11
4,True,False,None,Nuke,11
...,...,...,...,...,...
43229,False,False,None,Cache,-6
43230,False,False,None,Train,5
43231,True,False,None,Mirage,4
43232,False,False,None,Mirage,12


In [76]:
classifier_df = csgo_df.copy()
classifier_df = classifier_df.drop(['event_id', 'match_id', 'date', 'best_of'], axis=1)
classifier_df = classifier_df.drop(classifier_df.loc[:, '16_winner': '30_winner'].columns, axis=1)
classifier_df = classifier_df.drop(classifier_df.loc[:, '16_t1': '30_t1'].columns, axis=1)
classifier_df = classifier_df.drop(classifier_df.loc[:, '16_t2': '30_t2'].columns, axis=1)
classifier_df

,team_1,team_2,map,1_t1,2_t1,3_t1,4_t1,5_t1,6_t1,7_t1,...,8_winner,9_winner,10_winner,11_winner,12_winner,13_winner,14_winner,15_winner,win_ratio,t1_start_ct
0,G2,Natus Vincere,Nuke,4350.0,1100.0,22100.0,9350.0,25750.0,10400.0,24600.0,...,False,True,False,False,False,False,False,False,-12,False
1,G2,Natus Vincere,Dust2,3900.0,7400.0,23250.0,28500.0,31900.0,31700.0,18950.0,...,True,True,False,False,False,False,False,False,-3,True
2,G2,Natus Vincere,Mirage,4150.0,14300.0,2000.0,24800.0,9000.0,23150.0,21850.0,...,False,False,False,False,False,False,False,False,-14,False
3,Natus Vincere,Astralis,Dust2,4150.0,18050.0,21000.0,25850.0,25000.0,25000.0,27250.0,...,True,True,True,True,True,True,True,True,11,False
4,Natus Vincere,Astralis,Nuke,4200.0,10000.0,22000.0,24500.0,27550.0,29350.0,31950.0,...,True,True,True,True,True,True,True,True,11,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43229,FaZe,HellRaisers,Cache,4250.0,7400.0,17200.0,18500.0,26750.0,27450.0,26950.0,...,True,False,False,False,False,False,False,False,-6,False
43230,Astralis,CLG,Train,4250.0,1600.0,24250.0,17400.0,22050.0,6600.0,25650.0,...,True,True,True,True,True,False,False,False,5,False
43231,SK,TYLOO,Mirage,4350.0,7650.0,20000.0,21500.0,29150.0,26900.0,8750.0,...,False,False,True,True,False,False,True,True,4,True
43232,Virtus.pro,MVP Project,Mirage,4150.0,11800.0,1200.0,22950.0,25750.0,26850.0,27850.0,...,True,False,True,True,True,True,True,True,12,False
